In [13]:
from commodities import *
c = Commodity()
x = pd.DataFrame({"Copper":c.copper()['Price'], "Iron Ore": c.iron_ore()['Price'], "Aluminum": c.aluminum()['Price'], "Nickel": c.nickel()['Price']})
print(x.tail(24))

                  Copper    Iron Ore     Aluminum        Nickel
2019-12-01   6077.062500   90.977273  1771.380000  13829.425000
2020-01-01   6031.209091   95.217391  1773.090909  13506.863636
2020-02-01   5687.750000   87.625000  1688.095000  12715.550000
2020-03-01   5182.631818   88.659091  1610.890909  11846.227273
2020-04-01   5057.972000   83.750000  1459.934500  11804.009500
2020-05-01   5239.826316   91.333333  1466.371579  12179.605263
2020-06-01   5754.595455  103.340909  1568.570455  12727.152273
2020-07-01   6372.460870  108.021739  1643.812174  13402.303913
2020-08-01   6508.392857  120.071429  1738.555238  14575.782857
2020-09-01   6704.900000  123.000000  1743.774545  14857.488636
2020-10-01   6713.811364  119.250000  1806.101818  15239.363636
2020-11-01   7068.907143  123.523810  1935.283333  15807.733333
2020-12-01   7772.238095  153.065217  2014.671905  16823.038571
2021-01-01   7972.147500  166.738095  2003.975500  17863.176000
2021-02-01   8470.940000  162.325000  20

In [10]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

df = pd.DataFrame({"Copper":c.copper()['Price'], "Iron Ore": c.iron_ore()['Price'], "Aluminum": c.aluminum()['Price'], "Nickel": c.nickel()['Price']})

mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()

ef = EfficientFrontier(mu, S)
neutral = EfficientFrontier(mu, S, weight_bounds=(-1,1))
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('Copper', 0.51392), ('Iron Ore', 0.48608), ('Aluminum', 0.0), ('Nickel', 0.0)])
Expected annual return: 196.3%
Annual volatility: 86.2%
Sharpe Ratio: 2.25


(1.962817662564119, 0.861981204280245, 2.253897942225287)

In [15]:
from pypfopt import EfficientSemivariance
from pypfopt.expected_returns import mean_historical_return, returns_from_prices

df = pd.DataFrame({"Copper":c.copper()['Price'], "Iron Ore": c.iron_ore()['Price'], "Aluminum": c.aluminum()['Price'], "Nickel": c.nickel()['Price']})
mu = mean_historical_return(df)
historical_returns = returns_from_prices(df)

es = EfficientSemivariance(mu, historical_returns)
es.efficient_return(0.20)

weights = es.clean_weights()
print(weights)
es.portfolio_performance(verbose=True)

OrderedDict([('Copper', 0.13759), ('Iron Ore', 0.3382), ('Aluminum', 0.48765), ('Nickel', 0.03656)])
Expected annual return: 127.4%
Annual semi-deviation: 41.9%
Sortino Ratio: 2.99


(1.2736886456447096, 0.4188502270234596, 2.993166924019576)

In [16]:
import matplotlib as plt
from pypfopt import plotting

ef = EfficientFrontier(mu, S, weight_bounds=(None, None))
ef.add_constraint(lambda w: w[0] >= 0.2)
ef.add_constraint(lambda w: w[2] == 0.15)
ef.add_constraint(lambda w: w[3] + w[4] <= 0.10)

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.show()

IndexError: Index 4 is out of bounds for axis 0 with size 4.